# Step 1: mechanics with Euler-Lagrange

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from numpy import logical_and as npand
from numpy import logical_or as npor
import matplotlib

In [ ]:
from sympy import (symbols, pi, I, E, cos, sin, exp, tan, simplify, expand, factor, collect,
                   apart, cancel, expand_trig, diff, Derivative, Function, integrate, limit,
                   series, Eq, solve, dsolve, Matrix, N, preorder_traversal, Float, solve_linear_system,
                   eye, zeros, lambdify, Symbol)
from sympy.physics.mechanics import dynamicsymbols, init_vprinting

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML

In [ ]:
matplotlib.rcParams['animation.embed_limit'] = 200

In [ ]:
init_vprinting()

In [ ]:
m0, m1, l0, l1, t, g = symbols('m_0 m_1 l_0 l_1 t g')

In [ ]:
q0, q1 = dynamicsymbols('q_0 q_1')
q0, q1

In [ ]:
qdot0 = q0.diff(t)
qdot1 = q1.diff(t)
qdot0, qdot1

In [ ]:
qdotdot0 = qdot0.diff(t)
qdotdot1 = qdot1.diff(t)
qdotdot0, qdotdot1

In [ ]:
T = m0*l0**2*qdot0**2/2 + m1*(l0**2*qdot0**2 + l1**2*qdot1**2 + 2*l0*l1*qdot0*qdot1*cos(q1-q0))/2
T

In [ ]:
U = -m0*g*cos(q0)*l0 - m1*g*(l0*cos(q0)+l1*cos(q1))
U

In [ ]:
from optibot.symbolic import get_lagr_eqs, lagr_to_RHS, print_funcs

In [ ]:
lag_eqs = get_lagr_eqs(T, U, 2)
lag_eqs

In [ ]:
RHS = lagr_to_RHS(lag_eqs)
RHS

In [ ]:
import casadi as cas

In [ ]:
from optibot.casadi import RHS2casF

In [ ]:
F_cas = RHS2casF(RHS, 2)

In [ ]:
F_cas([1,0,1,1], [0,0], [1,1,1,1,1])

In [ ]:
F_cas

In [ ]:
from optibot.numpy import RHS2numpy

In [ ]:
F_nump = RHS2numpy(RHS, 2)

In [ ]:
F_nump([1,0,1,1], [0,0], [1,1,1,1,1])

In [ ]:
F_nump

In [ ]:
from optibot.symbolic import diff_to_symb

In [ ]:
E=T+U

print_funcs([diff_to_symb(E, 2),], 2)

In [ ]:
from optibot.numpy import unpack
def energ(x,params):
    q_0, q_1, v_0, v_1 = unpack(x)
    m_0, l_0, m_1, g, l_1 = params
    result = -g*l_0*m_0*np.cos(q_0)
    result += - g*m_1*(l_0*np.cos(q_0) + l_1*np.cos(q_1)) 
    result += l_0**2*m_0*v_0**2/2
    result += m_1*(l_0**2*v_0**2 + 2*l_0*l_1*v_0*v_1*np.cos(q_0 - q_1) + l_1**2*v_1**2)/2

    return result



In [ ]:
def create_anim(X, U):
    fig, ax = plt.subplots()

    fig.set_size_inches([8,8])
    ax.set_xlim(( -2.5, 2.5))
    ax.set_ylim(( -2.5, 2.5))

    circle1 = plt.Circle((0, 0), 2, color='b', ls = ":", fill=False)
    circle2 = plt.Circle((0, 0), 1, color='b', ls = ":", fill=False)
    ax.add_artist(circle1)
    ax.add_artist(circle2)

    line1, = ax.plot([], [], lw=2)
    line2, = ax.plot([], [], lw=2)
    line3, = ax.plot([], [], 'k', lw=1, ls = ':')
    point1, = ax.plot([], [], marker='o', markersize=15, color="red")
    point2, = ax.plot([], [], marker='o', markersize=15, color="red")
    text = ax.text(0.2, 0, "", fontsize = 12)
    text_2 = ax.text(0.2, -0.15, "", fontsize = 12)
    text_3 = ax.text(0.2, -0.30, "", fontsize = 12)
    
    trayectory = [[0,-2],]
    
    def init():
        line1.set_data([], [])
        line2.set_data([], [])
        line3.set_data([], [])
        point1.set_data([], [])
        point2.set_data([], [])
        text.set_text('')
        return (line1,line2,)
    def animate(i):
        x1 = [0, np.sin(X[i,0])]
        y1 = [0, -np.cos(X[i,0])]
        x2 = [x1[1], x1[1]+np.sin(X[i,1])]
        y2 = [y1[1], y1[1]-np.cos(X[i,1])]
        trayectory.append([x2[1], y2[1]])
        line1.set_data(x1, y1)    
        point1.set_data(x1[1], y1[1])
        line2.set_data(x2, y2)    
        point2.set_data(x2[1], y2[1])
        tray = np.array(trayectory)
        line3.set_data(tray[:,0], tray[:,1])    
        text.set_text("U = %.6f" % U[i,0])
        text_2.set_text(r"$\dot{\theta}_0$" + " = %.6f" % X[i,0])
        text_3.set_text(r"$\dot{\theta}_1$" + " = %.6f" % X[i,1])
        return (line1,line2,)
    N = X.shape[0]
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=N, interval=20, 
                               blit=True)
    return anim

In [ ]:
from optibot.schemes import euler_restr, trapz_restr, trapz_mod_restr, hs_mod_restr, hs_restr
from optibot.casadi import restriction2casadi

In [ ]:
N = 300

In [ ]:
new_X = np.zeros([N+1,4],dtype = 'float')
new_X[:,0]= np.linspace(0, cas.pi, N+1)
new_X[:,2]= np.linspace(0, cas.pi, N+1)
new_X[:,1]= cas.pi/N
new_X[:,3]= cas.pi/N
new_U = np.zeros([N+1,2],dtype = 'float')

In [ ]:
opti = cas.Opti()
opti.solver('ipopt')

In [ ]:
X = opti.variable(N+1,4)
U = opti.variable(N+1,2)
T = opti.parameter()
u_m = opti.parameter(2)

In [ ]:
#M = opti.parameter(2)
#L = opti.parameter(2)
#G = opti.parameter()
Params = opti.parameter(5)

In [ ]:
#cost = cas.sum1(2+cas.cos(X[:,0])+cas.cos(X[:,1]))*T
cost = cas.sum1(2+cas.cos(X[:,1]))*T
opti.minimize(cost)

In [ ]:
opti.subject_to(X[0,:].T == [0, 0, 0, 0])
opti.subject_to(cas.cos(X[-1,0]) < -0.9999)
opti.subject_to(cas.cos(X[-1,1]) < -0.9999)
opti.subject_to(opti.bounded(-0.001,X[-1,2],0.001))
opti.subject_to(opti.bounded(-0.001,X[-1,3],0.001))
opti.subject_to(U[-1,:].T == [0,0])

In [ ]:
#restriction = restriction2casadi(euler_restr, F_cas, 2, 5)
#restriction = restriction2casadi(trapz_restr, F_cas, 2, 5)
restriction = restriction2casadi(hs_restr, F_cas, 2, 5)

In [ ]:
for ii in range(N):
    opti.subject_to(restriction(X[ii,:], X[ii+1,:], U[ii,:], U[ii+1,:],T/N, Params)==0)
    opti.subject_to(opti.bounded(-u_m[0],U[ii, 0],u_m[0]))
    opti.subject_to(opti.bounded(-u_m[1],U[ii, 1],u_m[1]))

In [ ]:
opti.set_initial(X[:,0], np.linspace(0, cas.pi, N+1))
opti.set_initial(X[:,2], np.linspace(0, cas.pi, N+1))
opti.set_initial(X[:,1], cas.pi/N)
opti.set_initial(X[:,3], cas.pi/N)
opti.set_initial(U, np.zeros([N+1,2],dtype = 'float'))

In [ ]:
opti.set_initial(X, new_X)
opti.set_initial(U, new_U)

In [ ]:
opti.set_value(T, 60)
max_par = 0.4
opti.set_value(u_m, [max_par, 0])
opti.set_value(Params, [1,1,1,1,1])

In [ ]:
sol = opti.solve()

In [ ]:
xx = sol.value(X)
uu = sol.value(U)
ene = energ(xx, [1,1,1,1,1])
ene = np.array(ene)
pot = uu[:,0] * xx[:,2] * sol.value(T)/N
work = [ene[0]]
for ii in pot:
    work.append(work[-1] + ii)
work = np.array(work)    

In [ ]:
plt.figure(figsize=[14,10])
plt.plot(xx[:,0], 'b')
plt.plot(xx[:,2], 'b:')
plt.plot(xx[:,1], 'orange')
plt.plot(xx[:,3], 'orange', ls = ':')
plt.plot(uu[:,0], 'g:')
plt.plot(uu[:,1], 'r:')

In [ ]:
plt.figure(figsize=[14,10])
plt.plot(xx[:,2], 'orange')
plt.plot(ene.flatten(), 'r:')
plt.plot(uu[:,0], 'g:')
plt.plot(uu[:,0] * xx[:,2], 'b:')
plt.plot(work, 'b-', linewidth = 1)
plt.hlines([0,np.pi, -np.pi], 0, N, 'k', 'dotted')

In [ ]:
pot_ene = []
for ii in range(len(ene)-1):
    pot_ene.append(ene[ii+1]-ene[ii])
pot_ene = np.array(pot_ene)

In [ ]:
plt.figure(figsize=[14,10])
#plt.plot(xx[:,2], 'orange')
#plt.plot(ene.flatten(), 'r:')
plt.plot(pot, 'g:')
plt.plot(pot_ene, 'r:')
#plt.plot(-uu[:,0] * xx[:,2], 'b:')
#plt.plot(work, 'b-', linewidth = 1)
plt.hlines([0,], 0, N, 'k', 'dotted')

In [ ]:
new_X = xx
new_U = uu

In [ ]:
anim = create_anim(xx,uu)

In [ ]:
HTML(anim.to_jshtml())

In [ ]:
new_t_array = np.linspace(0, sol.value(T), 3*N)

In [ ]:
from optibot.schemes import interpolated_array

In [ ]:
def interpolated_array(X, U, F, h, t_array, params, scheme="hs_scipy"):
    N = t_array.size
    new_X = zeros(N)
    if X.shape[0] == U.shape[0] + 1:
        U = extend_array(U)
    if X.shape[0] != U.shape[0]:
        raise ValueError("X and U have incompatible sizes")
    old_t_array = np.linspace(0, (X.shape[0] - 1) * h, X.shape[0])
    if t_array[-1] - old_t_array[-1] > h * 1e-9:
        raise ValueError("Proposed time array extends outside interpolation")
    # print(Q.size, U.size, old_t_array.size, Q, U, old_t_array)
    new_U = np.interp(t_array, old_t_array, U)
    if scheme == "hs_scipy":
        X_interp = hermite(old_t_array, X, F(X, U, params))
        new_X = X_interp(t_array)
    else:
        for ii in range(N):
            new_X[ii] = newpoint(X, U, F, h, t_array[ii], params, scheme)
    return new_X, new_U


In [ ]:
new_X, new_U = interpolated_array(xx, uu, F_nump, sol.value(T)/N, new_t_array, sol.value(Params))

In [ ]:

np.interp(t_array, old_t_array, U)

In [ ]:
xx.shape

In [ ]:
uu.shape